# Scraping data from Xendan website

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm.notebook import tqdm


In [28]:
import urllib.parse

data_list = []
pages = set()

def getLinks(pageUrl):
    encoded_url = urllib.parse.quote(pageUrl, safe=':/?=&')
    html = urlopen(encoded_url)
    bs = BeautifulSoup(html, 'html.parser')
    
    header_elem = bs.find('div', {'class': "detail-top"})
    article_elem = bs.find('div', {'class': "detail-big-text-p"})
    
    if header_elem and article_elem and header_elem.find('h1'):
        header = header_elem.find('h1').text.strip()
        article = article_elem.text.strip()
        data_list.append({'Header': header, 'Article': article, 'Link': pageUrl})
        
    for link in bs.find_all('a', href=re.compile(r'https://www\.(xendan\.org/detailnews/\d+|xendan\.org/.*)')):
        if 'href' in link.attrs:
            newPage = link.attrs['href']
            if newPage not in pages and "babetakan.aspx" not in newPage and "multimedia.aspx" not in newPage and "jorawjor.aspx" not in newPage and "raport.aspx" not in newPage:
                print(newPage)
                pages.add(newPage)
                getLinks(urllib.parse.urljoin(pageUrl, newPage.replace('j', '')))

getLinks('https://www.xendan.org/detailnews/178566')


https://www.xendan.org/about.aspx?jimare=1
https://www.xendan.org/about.aspx?jimare=2
https://www.xendan.org/arshif.aspx?babet=kurd
https://www.xendan.org/sport
https://www.xendan.org/life
https://www.xendan.org/wtar.aspx
https://www.xendan.org/bzharda.aspx
https://www.xendan.org/detailnews/178611
https://www.xendan.org/detailnews/178610
https://www.xendan.org/detailnews/178609
https://www.xendan.org/detailnews/178608
https://www.xendan.org/detailnews/178606
https://www.xendan.org/detailnews/178604
https://www.xendan.org/detailnews/178603
https://www.xendan.org/detailnews/178602
https://www.xendan.org/detailnews/178601
https://www.xendan.org/detailnews/178600
https://www.xendan.org/detailnews/178013
https://www.xendan.org/detailnews/178378
https://www.xendan.org/detailnews/178581
https://www.xendan.org/detailnews/178573
https://www.xendan.org/detailnews/178555
https://www.xendan.org/detailnews/178546
https://www.xendan.org/detailnews/178071
https://www.xendan.org/detailnews/178258
http

In [30]:
scraped_data=pd.DataFrame(data_list)

## Using while loop to change the id of the news article tehn scraping it

In [3]:
data_list = []
partial_save_amount=5
def scrape_xendan(lastId):
    #using proxy to fetch data in case of bans or other issues like firewalls
    #proxy was created using nitro from simpleproxy github repository
    for i in tqdm(range(lastId, -1, -1)):
        url=f"https://www.xendan.org/detailnews/{i}"
        proxied_fetch=f'https://spontaneous-faloodeh-9e9321.netlify.app/?destination={url}'
        html = urlopen(proxied_fetch)
        bs = BeautifulSoup(html, 'html.parser')    
        header_elem = bs.find('div', {'class': "detail-top"})
        article_elem = bs.find('div', {'class': "detail-big-text-p"})
        img_elem=bs.find('div', {'class': "detail-big-img"})
        try:
            if header_elem and article_elem and header_elem.find('h1'):
                header = header_elem.find('h1').text.strip()
                article = article_elem.text.strip()
                image=img_elem.img['src'] if img_elem else " "
                data_list.append({'Header': header, 'Article': article, 'Link': url, 'Image': image})
        except:
            print(f'Error at {i}')
            continue

 
    
    


In [4]:
scrape_xendan(172550)

  0%|          | 0/172551 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [135]:
news_scraper=pd.DataFrame(data_list)

In [136]:
news_scraper.shape

(1244, 4)

In [137]:
news_scraper

,Header,Article,Link,Image
0,سەرکەوتنی ئاڵوگۆڕی دەستگیرکراوانی نێوان ئیسرائ...,خه‌ندان-\n\nوەزارەتی دەرەوەی قەتەر سەرکەوتنی ئ...,https://www.xendan.org/detailnews/173794,https://wenakan.xendan.org/root/root/images/20...
1,پنتاگۆن: لەدوای راگرتنی ئۆپەراسیۆنی غەززە هیچ ...,خه‌ندان-\n\nوه‌زاره‌تی به‌رگری ئه‌مریكا (پنتاگ...,https://www.xendan.org/detailnews/173793,https://wenakan.xendan.org/root/root/images/20...
2,له‌ روداوێكی هاتوچۆدا كه‌سێك گیانی له‌ده‌ستدا,خه‌ندان-\n\nوتەبێژی پۆلیسی پارێزگای دهۆک رایگه...,https://www.xendan.org/detailnews/173792,https://wenakan.xendan.org/root/root/images/19...
3,ئه‌مریكا هه‌ڵوێستی خۆی له‌باره‌ی گه‌ڕانه‌وه‌ی ...,خه‌ندان-\n\nوەزارەتی پێشمەرگە هەریەک لە بەرپرس...,https://www.xendan.org/detailnews/173791,https://wenakan.xendan.org/root/root/images/19...
4,تەندروستی جیهانی: خەڵکی غەززه‌ زیاتر مەترسیی م...,خه‌ندان-\n\nوتەبێژی رێکخراوی تەندروستی جیهانی ...,https://www.xendan.org/detailnews/173790,https://wenakan.xendan.org/root/root/images/18...
...,...,...,...,...
1239,هه‌موان چاوه‌ڕوانی وتاره‌كه‌ی نەسروڵان؛ ترسی ر...,خه‌ندان-\n\nئه‌مڕۆ هەینی حەسەن نەسروڵا، ئەمیند...,https://www.xendan.org/detailnews/172555,https://wenakan.xendan.org/root/root/images/63...
1240,چوار سەربازی ئیسرائیل لە غەززە کوژراون,خه‌ندان-\n\nئەمڕۆ سوپای ئیسرائیل کوژرانی چوار ...,https://www.xendan.org/detailnews/172554,https://wenakan.xendan.org/root/root/images/63...
1241,پێشنیازی ئاگربەست لە رۆژدا ده‌كرێت؛ ئەمریکا فش...,خه‌ندان-\n\nسەرچاوەیەکی فەڵەستینی ئاگادار لە و...,https://www.xendan.org/detailnews/172553,https://wenakan.xendan.org/root/root/images/63...
1242,ئەسەد ره‌زامه‌ندی له‌سه‌ر ناردنی سیستمی موشەکی...,خه‌ندان-\n\nبەشار ئەسەد، سەرۆکی سوریا رەزامەند...,https://www.xendan.org/detailnews/172552,https://wenakan.xendan.org/root/root/images/63...


In [138]:
news_scraper.to_csv('../scrapedData/xendan-5.csv')

In [119]:
img_dumb=[]
url=f"https://www.xendan.org/detailnews/{178140}"
proxied_fetch=f'https://spontaneous-faloodeh-9e9321.netlify.app/?destination={url}'
html = urlopen(proxied_fetch)
bs = BeautifulSoup(html, 'html.parser')    
header_elem = bs.find('div', {'class': "detail-top"})
article_elem = bs.find('div', {'class': "detail-big-text-p"})
img_elem=bs.find('div', {'class': "detail-big-img"})
#print(img_elem.img['src'])
if header_elem and article_elem and header_elem.find('h1'):
    header = header_elem.find('h1').text.strip()
    article = article_elem.text.strip()
    image=img_elem.img['src'] if img_elem else " "
    img_dumb.append({'Header': header, 'Article': article, 'Link': url})


In [115]:
img_dumb

[{'Header': 'ته\u200cقینه\u200cوه\u200cیه\u200cك له\u200c كفری رویدا و كه\u200cسێك گیانی له\u200cده\u200cستدا',
  'Article': 'خه\u200cندان-\n\nپاشنیوه\u200cڕۆی ئه\u200cمڕۆ ته\u200cقینه\u200cوه\u200cیه\u200cك له\u200cنزیك نه\u200cخۆشخانه\u200cی كفری رویدا و كه\u200cسێك گیانی له\u200cده\u200cستدا، سه\u200cرچاوه\u200cیه\u200cكی ئاگاداریش ده\u200cڵێت، هێرشه\u200cكه\u200c له\u200c رێگه\u200cی فڕۆكه\u200cی بێفڕۆكه\u200cوانه\u200cوه\u200c ئه\u200cنجامدراوه\u200c.\n\nبه\u200cگوێره\u200cی زانیاریه\u200cكانی خه\u200cندان، پاشنیوه\u200cڕۆی ئه\u200cمڕۆ یه\u200cكشه\u200cممه\u200c ته\u200cقینه\u200cوه\u200cیه\u200cك له\u200c نزیك بینایه\u200cك له\u200c نزیك نه\u200cخۆشخانه\u200cی كفری (ناوبازاڕی كفری) رویداوه\u200c به\u200cهۆیه\u200cوه\u200c هاوڵاتییه\u200cك به\u200cناوی عه\u200cلی حسێن ناسراو به\u200c (حسێن عەلوش) كوژراوه\u200c".\nئاگاداركرام كه\u200c له\u200c رێگه\u200cی فڕۆكه\u200cی بێفڕۆكه\u200cوانه\u200cوه\u200c مامم كراوه\u200cته\u200c ئامانج و گیانی له\u200cده\u200cستداوه\u200c\nبرازایه\u200c